# Data processing and linear regression with Exasol

This tutorial is for users who want to use Exasol as a data source for training a linear regression model. 
However, the data analyzing and processing part can also be interesting to anyone who's looking for information on how to prepare a dataset for training. 

In this tutorial, we will discuss the following topics:

- Part 1. How to import a dataset from CSV file to Exasol database.
- Part 2. How to analyze the data in the table.
- Part 3. How to prepare data for using it in a linear regression model.
- Part 4. How to create and train a model.

The model in this example will predict a flight's arriving delay.

### Prerequisites

The users are assumed to have a basic understanding of Exasol, SQL and basic Python programming knowledge.

## Part 1. Importing data

In our tutorial, we use `Flights.csv` file which provides information about U.S. domestic flights from 1987 to the 2019 year.
The file has more than 184 million rows and 109 columns.

As the CSV file is too big, - more than 80 GB - we can't work with it directly importing is as a Pandas DataFrame.
That's why we need to use a database as a warehouse and a workbench for a future data transformation process.

So, the first step: **we start an Exasol Database**. 
For this tutorial, we installed an Exasol Cloud Image using [Exasol Cloud Wizard](https://cloudtools.exasol.com/#/).
If you want to know more about Exasol Cloud Wizard, you can read an article about how to use this tool in our [blog](https://www.exasol.com/en/blog/building-clusters-in-the-sky-exasol-cloud-wizard/).
A local Exasol node was not a good option in our case because a local machine with this node proceeds data much slower than the cloud image.

The second step: **import data from CSV file to Exasol's table**. 

1. We create an SQL CREATE TABLE statement and put it in a separate file named `flights.sql`.
Our table will have 109 columns according to the CSV file. Here is a part of the query:

In [ ]:
CREATE OR REPLACE TABLE "FLIGHTS" ("YEAR" INTEGER, "QUARTER" INTEGER, "MONTH" INTEGER, "DAY_OF_MONTH" INTEGER, "DAY_OF_WEEK" DECIMAL(1,0), ... , ...);


2. We connect to Exasol, create a new schema and create a new table using the statement above.

 For connection and executing queries, we use [pyexasol](https://github.com/badoo/pyexasol) library.


In [ ]:
import pyexasol

connection = pyexasol.connect(dsn='host:port', user='username', password='password', compression=True)

connection.execute("CREATE SCHEMA IF NOT EXISTS {schema_name};".format(schema_name="FLIGHTS"))
connection.open_schema("FLIGHTS")
create_table_query = open('flights.sql', 'r')
for line in create_table_query:
    connection.execute(query=line)

 3. After the table is ready, we are importing the CSV file's content to the table using pyexasol. 
    The file is stored in Google Storage because of its size, so we only use a link to the file.

In [ ]:
connection.execute("IMPORT INTO {table_name} FROM CSV AT '{file_path}' FILE '{file_name}' "
            "COLUMN SEPARATOR = '{column_separator}' SKIP = 1  "
            "ERRORS INTO error_table (CURRENT_TIMESTAMP) REJECT LIMIT UNLIMITED ERRORS".format(
                table_name="FLIGHTS", file_path="https://storage.googleapis.com/our/path/", file_name="flights.single.csv.gz", column_separator=","))
connection.close()

Now you can find all the data in Exasol's table. 
Below you can see a python class that is doing the process described above.

In [ ]:
import simplestopwatch


class CsvImporter:
    def __init__(self, connection):
        self.connection = connection

    def import_file(self, sql_create_table_file, schema_name, table_name, file_path, file_name, column_separator):
        self.__handle_schema(self.connection, schema_name)
        self.__execute_create_table(sql_create_table_file)
        self.__run_import_command(self.connection, table_name, file_path, file_name, column_separator)

    def __execute_create_table(self, sql_create_table_file):
        query = open(sql_create_table_file, 'r')
        for line in query:
            self.connection.execute(query=line)

    def __handle_schema(self, connection, schema_name):
        connection.execute("CREATE SCHEMA IF NOT EXISTS " + schema_name + ";")
        connection.open_schema(schema_name)

    def __run_import_command(self, connection, table_name, file_path, file_name, column_separator):
        timer = simplestopwatch.Timer()
        connection.execute(
            "IMPORT INTO {table_name} FROM CSV AT '{file_path}' FILE '{file_name}' "
            "COLUMN SEPARATOR = '{column_separator}' SKIP = 1  "
            "ERRORS INTO error_table (CURRENT_TIMESTAMP) REJECT LIMIT UNLIMITED ERRORS".format(
                table_name=table_name, file_path=file_path, file_name=file_name, column_separator=column_separator))
        timer.stop()
        print("Imported in " + str(timer))

## Part 2. Analyzing data

Now when we have a dataset in the Exasol table, we can start analyzing data.

First, we should take a look at the columns list and decide which of them we would use for the model's training.

### How do we decide which columns we need?   

We just look at the columns one by one and judge whether it can be helpful for prediction or not.
A few examples of columns which we decided to use in our model for delay prediction:
- Day of week
- Departure airport
- Arrival airport, etc.

And also a few columns which we decided NOT to use:
- Flight number: a unique number that represents an exact flight.
- Delay reason: we don't need a reason to predict a delay.
- Canceled: this information doesn't affect delay.

### How to handle columns with similar information?

As we don't need the repeated information, we should select only one column which suits best to the model.
For example, we have three columns:
- Origin Airport: name of an airport as a string.
- Origin Airport ID: an identification number assigned by US DOT to identify a unique airport. 
- Origin Airport Sequence ID: an identification number assigned by US DOT to identify a unique airport at a given point in time.

These columns encode the same information but in different ways. We would select the second one - Origin Airport ID, for the model.
The first reason - it's numeric. And the second - this code is more stable than the Origin Airport Sequence ID as it can't be changed later.
The first reason - it's numeric. And the second - this code is more stable than the Origin Airport Sequence ID as it can't be changed later.

### Analyzing selected columns

The next step is to collect data about the content of the selected columns. 
Here is a small list which can give you an idea of how to analyze a column:

1. How many null values does the column contain?
2. How are the data represented in the column: string, number, date, etc?
3. How many distinct values does the column contain?
4. What are the maximum and minimum values (for numbers)?

For our example, we collected data using `pyexasol` and then created charts with `plotly` library.

In [ ]:
import plotly.express as plotly
import pyexasol
from plotly.graph_objs._figure import Figure
from pyexasol import ExaConnection


class ColumnStatisticCollector:
    def get_column_statistic(self, connection: ExaConnection, schema_name: str, table_name: str,
                             column_name: str) -> Figure:
        connection.open_schema(schema_name)
        sum_of_distinct_values = self.__get_query_result(connection,
                                                         'SELECT COUNT (DISTINCT "{column_name}") FROM {table_name}'
                                                         .format(column_name=column_name, table_name=table_name))
        sum_of_nulls = self.__get_query_result(connection,
                                               'SELECT COUNT(*) FROM  {table_name} WHERE "{column_name}" IS NULL'
                                               .format(table_name=table_name, column_name=column_name))
        max_value = self.__get_query_result(connection, 'SELECT MAX("{column_name}") FROM {table_name}'
                                            .format(column_name=column_name, table_name=table_name))
        min_value = self.__get_query_result(connection, 'SELECT MIN("{column_name}") FROM {table_name}'
                                            .format(column_name=column_name, table_name=table_name))

        result_set = connection.export_to_pandas(
            'SELECT DISTINCT "{column_name}", COUNT("{column_name}") AS sum_of_distinct_values FROM {table_name} '
            'GROUP BY "{column_name}" ORDER BY "{column_name}";'
                .format(column_name=column_name, table_name=table_name))

        bar = plotly.bar(result_set, x=(column_name), y="SUM_OF_DISTINCT_VALUES",
                         title="Column: " + column_name + ", sum of dist values=" + str(
                             sum_of_distinct_values) + ", nulls=" + str(sum_of_nulls) + ", max value=" + str(
                             max_value) + ", min value=" + str(min_value))
        bar.layout.xaxis.type = 'category'
        return bar

    def __get_query_result(self, connection: pyexasol.connection, query: str):
        iterable_query_result = connection.execute(query)
        counter = 0
        for row in iterable_query_result:
            counter = row[0]
        return counter

And here is an example of a chart:
<img src="img/img1.png">


## Part 3. Data pre-processing

After analyzing all the data, we created a new table containing only the columns which we consider can be used for the training and prediction of the linear regression.

### Cleaning data

In the new table we also made a few changes to clean the data:

- Removed a few rows where important values contain NULLs. 
  
  In our table, the data were relatively clean, and we removed less than 10 rows with NULL values of unknown origin.
  
- Replaces NULL values with new values. 
   
  For columns `origin state fips` and `destination state fips` we replaced NULLs with a new value which we can interpret as a 'place outside the USA' as the NULLs in the mentioned columns belonged to the cities not included into the USA.
  
  We also replace NULLs with 0 values for the `delay` column as we assumed that NULLs mean arriving without delay.

- Removed duplicated row.

  We consider the rows that contain the same a) `flight number`, b) `flight date`, c) `origin airport id`, d) `destination airport id`, e) `airline id` are duplicates. So we removed duplicated rows and left only one that has the highest value for `arriving delay`. 
  
  Here is an example of a query which looks for duplicates in the table: 

In [ ]:
SELECT * FROM TEST.FLIGHTS a
JOIN (SELECT "ORIGIN_AIRPORT_ID", COUNT("ORIGIN_AIRPORT_ID"), "FLIGHT_NUM", COUNT("FLIGHT_NUM"), "AIRLINE_ID", COUNT("AIRLINE_ID"), "DEST_AIRPORT_ID", COUNT("DEST_AIRPORT_ID"), "FLIGHT_DATE", COUNT("FLIGHT_DATE")
FROM TEST.FLIGHTS 
GROUP BY "ORIGIN_AIRPORT_ID", "FLIGHT_NUM", "AIRLINE_ID", "DEST_AIRPORT_ID", "FLIGHT_DATE"
HAVING (COUNT("ORIGIN_AIRPORT_ID") > 1) AND (COUNT("FLIGHT_NUM") > 1) AND (COUNT("AIRLINE_ID")>1) AND (COUNT ("DEST_AIRPORT_ID")>1) AND (COUNT("FLIGHT_DATE")>1) ) b
ON a."ORIGIN_AIRPORT_ID" = b."ORIGIN_AIRPORT_ID" AND a."FLIGHT_NUM" = b."FLIGHT_NUM" AND a."AIRLINE_ID" = b."AIRLINE_ID" AND a."DEST_AIRPORT_ID" = b."DEST_AIRPORT_ID" AND a."FLIGHT_DATE" = b."FLIGHT_DATE";

### Normalizing data

Before loading the data into the model, we need to normalize it. 
Data normalization is a process in which the data that has different ranges is cast to a certain interval, such as [0,1] or [-1,1]. 
Normalization can significantly increase the accuracy of the model. If normalization is not applied, it can lead to incorrect and inaccurate results.
There are a few different ways to normalize the data depending on the type of data the column contains.

We divided our columns into two groups depending on the types of values:

1. Numerical values with natural order — values that contain numbers in which sequence is important. 
    For example, for days of the month, it is important that the 29th comes before the 30th. In this group, we have the following columns: `year`, `month`, `day of month`, `day of week`, `delay`.

2. Categorical values — here we have values that can be grouped into a limited amount of categories.
    A few examples of the columns from this group: `origin airport id`, `origin city id`, `destination state fips` etc.

We are going to normalize the values of these two groups separately, and we can use different approaches depending on the amount of input data.

#### How to normalize a small dataset?

For a small dataset which can be stored in memory in full size, we can use a `sklearn.preprocessing` library to normalize the data.

For categorical data, we apply `LabelEncoder`. LabelEncoder is a utility class to help normalize labels such that they contain only values between 0 and N (number of unique categories - 1).
For numerical values with the natural order, we use `MinMaxScaler`. This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

We created a small utility class that encodes all the columns and creates a new table with encoded data for us.
Use this class when a table has no more than 10.000.000 rows.

In [ ]:
from typing import Any

import pandas
from pandas import DataFrame
from pyexasol import ExaConnection
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


class SmallDataTransformer:
    """
    Use this class when a table contains less than 10.000.000 rows.
    """

    def __init__(self, connection: ExaConnection):
        self.connection = connection

    def transform(self, schema_name: str, origin_table_name: str, final_table_name: str,
                  min_max_scaling_columns_list: list,
                  categorical_columns_list: list, all_columns_with_types_list: str) -> None:
        self.connection.open_schema(schema_name)
        pandas_result_set = self.connection.export_to_pandas(
            "SELECT * FROM {table_name}".format(table_name=origin_table_name))
        encoded_categorical_columns_data_frame = self.__encode_as_labels(categorical_columns_list, pandas_result_set)
        encoded_numerical_columns_data_frame = pandas.DataFrame.from_records(
            self.__encode_as_max_min(min_max_scaling_columns_list,
                                     pandas_result_set))
        all_columns_data_frame = pandas.concat(
            [encoded_categorical_columns_data_frame, encoded_numerical_columns_data_frame],
            axis=1)
        self.connection.execute(
            'CREATE OR REPLACE TABLE {final_table_name} ({all_columns_with_types_list});'
                .format(final_table_name=final_table_name, all_columns_with_types_list=all_columns_with_types_list))
        self.connection.import_from_pandas(all_columns_data_frame, final_table_name)

    def __encode_as_labels(self, categorical_columns_list: list, columns_data_frame: DataFrame) -> Any:
        if len(categorical_columns_list) != 0:
            categorical_columns_data_frame = columns_data_frame[categorical_columns_list]
            if not categorical_columns_data_frame.empty:
                return categorical_columns_data_frame.apply(LabelEncoder().fit_transform)
        else:
            return None

    def __encode_as_max_min(self, min_max_scaling_columns_list: list, columns_data_frame: DataFrame):
        if len(min_max_scaling_columns_list) != 0:
            min_max_columns_data_frame = columns_data_frame[min_max_scaling_columns_list]
            if not min_max_columns_data_frame.empty:
                return MinMaxScaler().fit_transform(min_max_columns_data_frame)
        else:
            return None


#### How to normalize a big dataset?

For a bigger dataset, it is preferable to use SQL directly for pre-processing data.
So we are going to apply the same `min-max scaling` and `label encoding` techniques, but to do it without ready-to-use libraries.

1. Min-max scaling 

    To apply the `min-max scaling` to the column we need to find the minimum and maximum values for the whole dataset, but then we can use scaling to batches of any size. 
    For example, we use the following query to encode the first 100 lines of the column `day of week`.
    

In [ ]:
SELECT 1.00 * ("DAY_OF_WEEK" - MIN("DAY_OF_WEEK") OVER()) / (MAX("DAY_OF_WEEK") OVER () - MIN("DAY_OF_WEEK") OVER ()) FROM TEST.FILTERED_FLIGHTS LIMIT 100;

2. Label encoding
    
    With the following two steps we apply the label encoding to a categorical column: 

    1. We find all distinct values in the column and assign ids to them starting from 0. In our case, we saved this data to a new table.
    2. We replace all original values with ids.


In [ ]:
CREATE OR REPLACE TABLE TEST.DEST_AIRPORT_ID_CATEGORIES AS SELECT rownum - 1 AS id, DEST_AIRPORT_ID FROM (SELECT DISTINCT DEST_AIRPORT_ID FROM TEST.FILTERED_FLIGHTS);
 
SELECT DEST_AIRPORT_ID_CATEGORIES.id, FILTERED_FLIGHTS.DEST_AIRPORT_ID FROM TEST.FILTERED_FLIGHTS JOIN TEST.DEST_AIRPORT_ID_CATEGORIES ON FILTERED_FLIGHTS.DEST_AIRPORT_ID = DEST_AIRPORT_ID_CATEGORIES.DEST_AIRPORT_ID;

As encoding of columns one by one takes a lot of time, we created a python script that generates a query, encodes all the columns and writes them into a new table.


In [ ]:
from pyexasol import ExaConnection


class LargeDataTransformer:
    def __init__(self, connection: ExaConnection):
        self.connection = connection

    def transform(self, schema_name: str, origin_table_name: str, new_table_name: str,
                  min_max_scaling_column_list: list,
                  categorical_columns_list: list):
        self.connection.open_schema(schema_name)
        select_subquery = self.__create_select_subquery(origin_table_name,
                                                        categorical_columns_list, min_max_scaling_column_list)
        query = "CREATE OR REPLACE TABLE {new_table_name} AS {select_subquery}".format(new_table_name=new_table_name,
                                                                                       select_subquery=select_subquery)
        print(query)
        self.connection.execute(query)

    def __create_select_subquery(self, origin_table_name: str,
                                 categorical_columns_list: list, min_max_scaling_column_list: list) -> str:
        select_subquery = "SELECT {max_min_scaling_columns_query}{comma}{categorical_columns_query} " \
                          "FROM {origin_table_name} {categorical_columns_postfix}"
        max_min_scaling_columns_query = self.__generate_query_for_min_max_scaling_columns(min_max_scaling_column_list)
        comma = ", " if len(max_min_scaling_columns_query) > 0 and len(categorical_columns_list) > 0 else ""
        categorical_columns_query = ""
        categorical_columns_postfix = ""
        if len(categorical_columns_list) > 0:
            categories_table_name_postfix = '_CATEGORIES'
            self.__create_categorical_tables(origin_table_name, categorical_columns_list, categories_table_name_postfix)
            categorical_columns_query = self.__generate_query_for_categorical_columns(categorical_columns_list,
                                                                                      categories_table_name_postfix)
            categorical_columns_postfix = self.___generate_categorical_columns_postfix(origin_table_name,
                                                                                       categorical_columns_list,
                                                                                       categories_table_name_postfix)
        return select_subquery.format(max_min_scaling_columns_query=max_min_scaling_columns_query,
                                      comma=comma,
                                      categorical_columns_query=categorical_columns_query,
                                      origin_table_name=origin_table_name,
                                      categorical_columns_postfix=categorical_columns_postfix)

    def __generate_query_for_min_max_scaling_columns(self, min_max_scaling_column_list):
        individual_columns_queries = []
        for column in min_max_scaling_column_list:
            individual_columns_queries.append(self.__generate_min_max_scale_column_query(column))
        return ', '.join(individual_columns_queries)

    def __generate_min_max_scale_column_query(self, column_name: str) -> str:
        return '1.00 * ("{column_name}" - MIN("{column_name}") OVER()) / (MAX("{column_name}") OVER () - MIN("{column_name}") OVER ()) AS "{column_name}"'.format(
            column_name=column_name)

    def __create_categorical_tables(self, origin_table_name: str,
                                    categorical_columns_list: list, categories_table_name_postfix: str):
        for column_name in categorical_columns_list:
            categories_table_name = column_name + categories_table_name_postfix
            self.connection.execute(
                'CREATE OR REPLACE TABLE {categories_table_name} AS SELECT rownum - 1 AS id, {column_name} FROM (SELECT DISTINCT {column_name} FROM {origin_table_name})'
                    .format(categories_table_name=categories_table_name, column_name=column_name,
                            origin_table_name=origin_table_name))

    def __generate_query_for_categorical_columns(self, categorical_columns_list: list,
                                                 categories_table_name_postfix: str) -> str:
        individual_columns_queries = []
        for column_name in categorical_columns_list:
            table_name = column_name + categories_table_name_postfix
            individual_columns_queries.append('{table_name}.id AS {column_name}'.format(table_name=table_name,
                                                                                        column_name=column_name))
        return ', '.join(individual_columns_queries)

    def ___generate_categorical_columns_postfix(self, origin_table_name: str,
                                                categorical_columns_list: list,
                                                categories_table_name_postfix: str) -> str:
        individual_columns_prefixes = []
        for column_name in categorical_columns_list:
            table_name = column_name + categories_table_name_postfix
            individual_columns_prefixes.append(
                "JOIN {table_name} ON {origin_table_name}.{column_name} = {table_name}.{column_name}".format(
                    column_name=column_name, table_name=table_name, origin_table_name=origin_table_name))
        return ' '.join(individual_columns_prefixes)
